Note: I ran my file in colab. But due to some technical issues, I was unable to upload it via colab. So I have downloaded a copy in my vs code and i am uploading it from there. But all the settings are as per colab.


Referenced Sites:
* https://docling-project.github.io/docling/examples/hybrid_chunking/#overview

* https://www.youtube.com/watch?v=9lBTS5dM27c&list=WL&index=2&t=1s

* Contextual Chunking-> https://www.anthropic.com/news/contextual-retrieval

* LanceDB embeddings-> https://lancedb.github.io/lancedb/embeddings/default_embedding_functions/#text-embedding-functions

I am doing text embeddings only. Not multi-modal embeddings.

* Building with Google's gemini embeddings-> https://lancedb.github.io/lancedb/embeddings/available_embedding_models/text_embedding_functions/gemini_embedding/

In [1]:
! pip install docling

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.0/186.0 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 17.7 MB/s eta 0:00:00
   ━━━━

In [2]:
from docling.document_converter import DocumentConverter
converter= DocumentConverter()

result= converter.convert('combined_markdown.md')
result= result.document

In [3]:
! pip install huggingface_hub

In [4]:
from huggingface_hub import login
from google.colab import userdata
token= userdata.get('HF_TOKEN_ORIGINAL_AGENTCOURSE')
login(token=token)

Setting up the configurations of bitsAndBytes config.

Note-> the latest version ,when i made this notebook, of bitsandbytes==0.46 is supported by only cuda==12.3. while the latest version of cuda is 12.4.
So i made the following changes. make these changes only after a proper research.

In [ ]:
# 1. Clean broken bitsandbytes installs
!pip uninstall -y bitsandbytes
!rm -rf /usr/local/lib/python*/dist-packages/bitsandbytes*

# 2. Set the manual override for CUDA 12.3
import os
os.environ["BNB_CUDA_VERSION"] = "123"

# 3. Reinstall the latest bitsandbytes (supports 12.3)
!pip install bitsandbytes
!pip install -U transformers accelerate

# 4. Verify CUDA backend
!python -m bitsandbytes


In [ ]:
# loading the model for tokenizing

from transformers import BitsAndBytesConfig, Gemma3ForCausalLM

bnb_config = BitsAndBytesConfig(load_in_8bit=True)

model = Gemma3ForCausalLM.from_pretrained(
    "google/gemma-3-1b-it",
    quantization_config=bnb_config,
    device_map="auto"
)


In [ ]:
# let us work on the chunking using docling
!pip install -qU pip docling transformers

In [ ]:
# if you want to use the hugging face tokenizers
!pip install 'docling-core[chunking]'

In RAG it is important to make sure that the chunker and the embedding model are using the same tokenizer.

okay so currently i do not see

In [ ]:
# lets build our tokenizer
from docling_core.transforms.chunker.tokenizer.huggingface import HuggingFaceTokenizer
from transformers import AutoTokenizer
model_id= "google/gemma-3-1b-it"

max_tokens= 400

tokenizer= HuggingFaceTokenizer(
    tokenizer=AutoTokenizer.from_pretrained(model_id),
    max_tokens=max_tokens
)

In [10]:
# now we can build our chunker
from docling.chunking import HybridChunker

chunker= HybridChunker(
    tokenizer=tokenizer,
    merge_peers= True # merges the undersized chunks that share the same headings or metadata
)

def build_and_get_chunks(doc):
  chunk_iter= chunker.chunk(doc)
  chunks= list(chunk_iter)

  return chunks

In [11]:
chunks= build_and_get_chunks(result)

we want to embed the contextualized chunkings. so:

In [12]:
def get_contextualized_chunks(chunks):
  contextualized_chunks= []
  for chunk in chunks:
    enriched_text= chunker.contextualize(chunk=chunk)
    contextualized_chunks.append(enriched_text)
  return contextualized_chunks

In [13]:
contextualized_chunk_list= get_contextualized_chunks(chunks)

In [14]:
contextualized_chunk_list[100]

"Dear\tRon,\tand\tHarry\tif\tyou're\tthere,\nDraco\tturned\taway\tand\tsaw\tthe\tcabinet\tright\tin\tfront\tof\thim.\tHe\twalked forward\t…\the\tstretched\tout\this\thand\tfor\tthe\thandle\t…\n'Done,'\tsaid\tMr\tMalfoy\tat\tthe\tcounter.\t'Come,\tDraco!'\nHarry\twiped\this\tforehead\ton\this\tsleeve\tas\tDraco\tturned\taway.\n'Good\tday\tto\tyou,\tMr\tBorgin,\tI'll\texpect\tyou\tat\tthe\tmanor\ttomorrow\tto pick\tup\tthe\tgoods.'\nThe\tmoment\tthe\tdoor\thad\tclosed,\tMr\tBorgin\tdropped\this\toily\tmanner.\n'Good\tday\tyourself, Mister Malfoy,\tand\tif\tthe\tstories\tare\ttrue,\tyou\thaven't sold\tme\thalf\tof\twhat's\thidden\tin\tyour manor …'\nMuttering\tdarkly,\tMr\tBorgin\tdisappeared\tinto\ta\tback\troom.\tHarry\twaited for\ta\tminute\tin\tcase\the\tcame\tback,\tthen,\tquietly\tas\the\tcould,\tslipped\tout\tof\tthe cabinet,\tpast\tthe\tglass\tcases\tand\tout\tof\tthe\tshop\tdoor."

In [15]:
! pip install -U -q 'google-genai'

In [16]:
from google import genai

client=genai.Client(api_key=userdata.get('GOOGLE_2_API_KEY'))

In [ ]:
! pip install chromadb

In [24]:
from google.genai import types
from chromadb import Documents, EmbeddingFunction, Embeddings


# creating a cutsom embedding function, that plugs into chromadb

class GeminiEmbeddingFunction(EmbeddingFunction):
  def __init__(self):
    # embedding function is an abstract base class in chromadb
    pass
    # required by chromadb


# overwrites the __call__ func() of chromadb, to generate embeddings on demand
  def __call__(self, input:Documents) -> Embeddings:
    Embedding_Model_ID= "models/text-embedding-004"

    # ensuring input is always a list, even if a single string.
    if isinstance(input, str):
      input= [input]

    all_embeddings=[]
    batch_size=100

    for i in range(0, len(input), batch_size):
      batch= input[i:i+batch_size]
      try:
        response= client.models.embed_content(
            model=Embedding_Model_ID,
            contents= batch,
            config=types.EmbedContentConfig(
                task_type='retrieval_document',

            )
        )
        # return all teh embeddings
        batch_embeddings= [emb.values for emb in response.embeddings]
        all_embeddings.extend(batch_embeddings)

      except Exception as e:
        print(f"Embedding error:{e}")
        return None
    return all_embeddings

In [25]:
# fix chromadb for batch processing
import chromadb
def create_chroma_db(contextualized_chunk_list):
  chroma_client= chromadb.PersistentClient(path="novel_collection_embeddings_3")

  collection= chroma_client.get_or_create_collection(
      name='bing_novelrag_collection_3',
      embedding_function=GeminiEmbeddingFunction()
  )
  print(f"Adding {len(contextualized_chunk_list)} chunks to database...")

  # add documents in batches to avoid memory issues
  batch_size=50

  for i in range(0, len(contextualized_chunk_list), batch_size):
    end_idx= min(i+batch_size, len(contextualized_chunk_list))
    batch_docs= contextualized_chunk_list[i:end_idx]
    batch_ids= [str(j) for j in range(i, end_idx)]


    # triggers embedding via geminiembeddingfunction call
    collection.add(
        documents= batch_docs,
        ids= batch_ids
    )

    print(f"Added batch {i//batch_size + 1}/{(len(contextualized_chunk_list)+ batch_size - 1)//batch_size}")


  print(f"Database created with {collection.count()} documents")

  return collection


In [26]:
# Set up the DB
collection_built = create_chroma_db(contextualized_chunk_list)

Adding 1346 chunks to database...
Added batch 1/27
Added batch 2/27
Added batch 3/27
Added batch 4/27
Added batch 5/27
Added batch 6/27
Added batch 7/27
Added batch 8/27
Added batch 9/27
Added batch 10/27
Added batch 11/27
Added batch 12/27
Added batch 13/27
Added batch 14/27
Added batch 15/27
Added batch 16/27
Added batch 17/27
Added batch 18/27
Added batch 19/27
Added batch 20/27
Added batch 21/27
Added batch 22/27
Added batch 23/27
Added batch 24/27
Added batch 25/27
Added batch 26/27
Added batch 27/27
Database created with 1346 documents


In [27]:
collection_built.peek()

{'ids': ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'],
 'embeddings': array([[-0.03861213,  0.02344161, -0.00374704, ...,  0.00964072,
          0.0244747 , -0.01426351],
        [-0.02072495,  0.02093361, -0.00411656, ...,  0.05033953,
         -0.00417255, -0.02308157],
        [-0.0383333 ,  0.01372326, -0.02511429, ...,  0.06165662,
          0.01992766,  0.00818526],
        ...,
        [-0.03405032,  0.00664998, -0.00151566, ...,  0.0002513 ,
          0.04247424, -0.03012509],
        [-0.05485309, -0.00544187,  0.01287586, ..., -0.02370497,
          0.02652791, -0.02154174],
        [-0.03307312, -0.00749543, -0.02514764, ...,  0.01036208,
         -0.01646801, -0.03018571]]),
 'documents': ['HARRY POTTER\nand\tthe\tChamber\tof\tSecrets\nJ.K.\tROWLING\nAll\trights\treserved;\tno\tpart\tof\tthis\tpublication\tmay\tbe\treproduced\tor\ttransmitted\tby\tany\tmeans, electronic,\tmechanical,\tphotocopying\tor\totherwise,\twithout\tthe\tprior\tpermission\tof\tthe\tpublisher\nTh

In [28]:
def get_relevant_passage(query, db, n_results=3):
  """Get relevant passages with better error handling"""

  try:
    results=db.query(query_texts=[query], n_results=n_results)
    if not results['documents']:
      print("No results found")
      return None
    return results['documents'][0]

  except Exception as e:
    print(f"Error in query: {e}")
    return None

In [49]:
# Perform embedding search
passage = get_relevant_passage("What are muggles?", collection_built)
print(passage)

["'The what? '\n'It's\tall\tto\tdo\twith\tbewitching\tthings\tthat\tare\tMuggle-made,\tyou\tknow,\tin case\tthey\tend\tup\tback\tin\ta\tMuggle\tshop\tor\thouse.\tLike,\tlast\tyear,\tsome\told witch\tdied\tand\ther\ttea\tset\twas\tsold\tto\tan\tantiques\tshop.\tThis\tMuggle\twoman bought\t it,\t took\t it\t home\t and\t tried\t to\t serve\t her\t friends\t tea\t in\t it.\t It\t was\t a nightmare\t-\tDad\twas\tworking\tovertime\tfor\tweeks.'", "Owl\tPost\nNon-magic people (more commonly known as Muggles) were particularly\t afraid\t of\t magic\t in\t medieval\t times,\t but\t not\t very\t good\t at recognising\tit.\tOn\tthe\trare\toccasion\tthat\tthey\tdid\tcatch\ta\treal\twitch\tor wizard,\t burning\t had\t no\t effect\t whatsoever.\t The\t witch\t or\t wizard\t would perform\ta\tbasic\tFlame-Freezing\tCharm\tand\tthen\tpretend\tto\tshriek\twith pain\t while\t enjoying\t a\t gentle,\t tickling\t sensation.\t Indeed,\t Wendelin\t the Weird\tenjoyed\tbeing\tburnt\tso\tmuch\tthat\tshe\tall

In [30]:
!curl https://ollama.ai/install.sh | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13281    0 13281    0     0  54192      0 --:--:-- --:--:-- --:--:-- 54430
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [31]:
!nohup ollama serve > ollama.log 2>&1 &

In [36]:
! ollama run gemma3n:latest “What is the capital of the Netherlands?”

The capital of the Netherlands is **Amsterdam**. 

While The Hague is the seat of government, Amsterdam is the capital city. 😊 




In [34]:
! pip install ollama

In [50]:
# send the query to ollama gemma3n model
import ollama
def send_response_to_ollama(response, model_name='gemma3n:latest'):
  client= ollama.Client()

  prompt= f"""You are a Harry Potter expert specializing in the first three books: Philosopher's Stone, Chamber of Secrets, and Prisoner of Azkaban.

  Your task: A user asked a question about Harry Potter, and our system retrieved the following relevant passages from the novels. Based on these passages, provide a comprehensive answer that directly addresses what the user likely asked.

  Guidelines:
  - Write a clear, informative answer in 50-150 words
  - Focus on the key information from the retrieved passages
  - Maintain the magical tone of the Harry Potter universe
  - If multiple concepts are mentioned, prioritize the most relevant ones
  - Don't speculate beyond what's provided in the retrieved text


  Retrieved Information:
  {response}

  Summary: """

  try:
    response= ollama.chat(
        model= model_name,
        messages=[
            {
                'role': 'user',
                'content': prompt,
            }
        ],
        options={
            'temperature': 0.3,
            'max_tokens': 250,
            'top_p': 0.5
        }

    )
    return response['message']['content']
  except Exception as e:
    print(f"Error in ollama chat: {e}")
    return None



In [51]:
final_response= send_response_to_ollama(passage)

In [52]:
final_response

"The passages reveal intriguing details about the world of magic and its interactions with the Muggle world. It seems that Muggles, or non-magic people, often fear magic, though they aren't adept at recognizing it. Attempts to harm wizards or witches with fire are futile, as they can employ a basic Flame-Freezing Charm. \n\nFurthermore, there's a concern about magical objects ending up in Muggle possession, as illustrated by the story of an old witch's tea set. Harry Potter is shown carefully writing with quill and ink, taking precautions to avoid being discovered by the Dursley family. \n\nThese snippets offer a glimpse into the complexities of a world where magic and the mundane coexist, with its own set of rules and hidden dangers.\n\n\n\n"